In [82]:
# Useful Libraries - https://github.com/codelucas/newspaper

In [174]:
#Importing Dependencies
import requests
from urllib.parse import urljoin, urlunsplit, urlparse
import bs4
from bs4 import BeautifulSoup
from bs4.element import Comment
from collections import Counter
from string import punctuation
import csv
import pandas as pd


#Tokenization Of Sentences
import nltk
nltk.download('punkt')

#Readability Scores
import textstat

In [321]:

urls = ['https://www.moneysavingexpert.com/travel/travel-tips/', 
        'https://www.moneysavingexpert.com/deals/deals-hunter/2018/05/how-to-get-free-cat-treats-and-other-pet-tips/', 
       'https://www.smartinsights.com/internet-advertising/internet-advertising-analytics/display-advertising-clickthrough-rates/'
       ]


In [378]:
class Crawler():
    def __init__(self, *args):
        self.urls = urls
        self.master_dict = {
            'URL':[],
            'Title_Text': [],
            'Title_Tag_Length': [],
            'Meta_Description': [],
            'Meta_Description_Length': [],
            'Content': [],
            'Word_Length': [],
            'Number_Of_Links': [],
            'Links_To_Text_Ratio': [],
            'Number_Of_Images':[],
            'H1_text': [],
            'H1_length': [],
            'H2_text': [],
            'H2_length': [],    
            'H3_text': [],
            'H3_length': [],    
            'H4_text': [],
            'H4_length': [],
            'H5_text': [],
            'H5_length': [],
            'lexicon_count': [],
            'Setences_Text': [],
            'Number_Of_Sentences': [],
            'Flesch_Reading_Ease_formula': [],
            'Flesch_Kincaid_Grade_Level': [],
            'FOG_Scale': [],
            'SMOG_Index': [],
            'ARI_Index': [],
            'Page_Size_In_Bytes': [],
            'Plain_Text_Size': [],
            'PLain_Text_Rate': [],
            'Encoding_Type': [],
            'Status_Code':[],
            'seo_friendly_url_characters_check': [],
            'seo_friendly_url_relative_length_check': [],
            'SSL/HTTPS': []
            
            ### Potential User Metrics ###
            #1.  User Comments (Text) 
            #2. Number of User Comments
            #3. User Comments Sentiment

            ### Page Speed Metrics ###
            #1. Meaningful Paint
            #2. Time To Interactive

            ### Link Metrics ###
            #1. DR 
            #2. PageURL Ranking 
            #3. Number of Referring Domains To Page
            #4. Number of Backlinks To Page
    
}        
        
        
    def get_metadata(self, soup):
        #Get Title Tag Of Page
        title_text = soup.title.getText()
        #Title Tag Length In Characters
        title_tag_length = len(title_text)
        # First get the meta description tag
        description = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})
        # If description meta tag was found, then get the content attribute and save it to db entry
        if description:
            meta_description = description.get('content') 
        return title_text, title_tag_length, meta_description
    
    
    def extract_page_features(self, soup):
    # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()    # rip it out

        # get text
        text = soup.body.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        #Word Length of The Webpage
        word_length = len(text)

        #Number of Links
        body_links = soup.body.find_all('a')
        number_of_links = len(body_links)

        #Links To Text Ratio
        links_to_text_ratio = len(body_links) / word_length

        #Number of Images
        Number_of_images = len([imgtag for imgtag in soup.body.find_all('img')])

        return text, word_length, body_links, number_of_links, links_to_text_ratio, Number_of_images
        
    
    def crawl(self, *args):
        for url in self.urls:
            
            #Loading BeautifulSoup
            req = requests.get(url)
            response = req.text
            soup = BeautifulSoup(response)
            
            #Meta_data
            meta_data = self.get_metadata(soup)
            self.master_dict['Title_Text'].append(meta_data[0])
            self.master_dict['Title_Tag_Length'].append(meta_data[1])
            self.master_dict['Meta_Description'].append(meta_data[2])
            
            #Extracting_Text_Features
            text_features = self.extract_page_features(soup)
            print(text_features)
            
            
            

In [379]:
crawler = Crawler(urls)

In [382]:
soup = BeautifulSoup(response)

In [383]:
#Get Title Tag Of Page
title_text = soup.title.getText()

#Title Tag Length In Characters
title_tag_length = len(title_text)

In [384]:
# First get the meta description tag
description = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})

# If description meta tag was found, then get the content attribute and save it to db entry
if description:
    meta_description = description.get('content') 

In [83]:
#Extracting Visible Text From The Webpage

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.body.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [84]:
#Word Length of The Webpage
word_length = len(text)

In [115]:
#Number of Links
body_links = soup.body.find_all('a')
number_of_links = len(body_links)  

In [117]:
number_of_links

353

In [93]:
#Links To Text Ratio
links_to_text_ratio = len(body_links) / word_length

In [103]:
#Number of Images
Number_of_images = len([imgtag for imgtag in soup.body.find_all('img')])

In [123]:
#Create A DataFrame Here!!!

df = pd.Series({'Title_Tag': title_text, 
                   'Title_Tag_Length':title_tag_length, 
                   'Meta_Description':meta_description,
                   'Body_Content_Text':text,
                   'Word_Length': word_length,
                   '<a>_Links_In_Body': number_of_links,
                   'Links_To_Text_Ratio': links_to_text_ratio,
                   'Number_Of_Images': Number_of_images,
                   'URL': test_url
    
})

In [124]:
df

Title_Tag                           Submit Listing | Eastern Daily Press
Title_Tag_Length                                                      36
Meta_Description       News, sport, entertainment and community infor...
Body_Content_Text      Register\nLog in | Register\nJobs24\nLocalsear...
Word_Length                                                         6241
<a>_Links_In_Body                                                    353
Links_To_Text_Ratio                                            0.0565614
Number_Of_Images                                                      18
URL                     https://www.edp24.co.uk/going-out/submit-listing
dtype: object

In [148]:
def dummy_variables(url):
    
    if 'blog' in url:
        df['Blog_In_URL'] = 1
    else: 
        df['Blog_In_URL'] = 0    
            
    if 'community' in url:
        df['Community_In_URL'] = 1
    else: 
        df['Community_In_URL'] = 0
        
    if ('directory' in url) or ('listing' in url):
        df['Directory'] = 1
    else: 
        df['Directory'] = 0
        
    if ('links' in url) or ('resources' in url):
        df['Links_In_URL'] = 1
    else: 
        df['Links_In_URL'] = 0


In [146]:
dummy_variables(df['URL'])

In [167]:
#Header Code

h1_text = soup.h1
number_of_h1 = len(soup.h1)

h2_text = soup.h2.getText()
number_of_h2 = len(soup.h2)

h3_text = soup.h3.getText()
number_of_h3 = len(soup.h3)

h4_text = soup.h4.getText()
number_of_h4 = len(soup.h4)


In [170]:
df['h1_text'] = h1_text
df['h2_text'] = h2_text
df['h3_text'] = h3_text
df['h4_text'] = h4_text

[nltk_data] Downloading package punkt to C:\Users\james.DESKTOP-36N7TS
[nltk_data]     Q\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [187]:
sentences = nltk.sent_tokenize(text)

In [188]:
test_url

'https://www.edp24.co.uk/going-out/submit-listing'

In [193]:
#Readability Scores For A Given Textual Input

lexicon_count = textstat.lexicon_count(text, removepunct=True)
number_of_sentences = len(sentences)
Flesch_Reading_Ease_formula = textstat.flesch_reading_ease(text)
Flesch_Kincaid_Grade_Level = textstat.flesch_kincaid_grade(text)
FOG_Scale = textstat.gunning_fog(text)
SMOG_Index = textstat.smog_index(text)
ARI_Index = textstat.automated_readability_index(text)

In [210]:
#Page_Size_In_Bytes
page_size_in_bytes = len(req.content)

#Plain_text_size
plain_text_size = len(text)

#plain_text_rate --> plaintext rate value (plain_text_size / page_size)
plain_text_rate = (plain_text_size / page_size_in_bytes) * 100 

In [213]:
#Encoding 
encoding = req.encoding

'utf-8'

In [214]:
#Status_Code
status_code = req.status_code

seo_friendly_url_characters_check -- checking for symbols in accordance with Google recommendations only uppercase and lowercase Latin characters, digits and dashes are allowed ‘true’ - if the test is passed.

In [248]:
seo_list = ascii_letters + ascii_lowercase + ascii_uppercase + digits + extra_chars
ascii_letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
ascii_lowercase = 'abcdefghijklmnopqrstuvwxyz'
ascii_uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
digits = '0123456789'
extra_chars = '.|-'
    
new_url = []

for character in awesome:
    if character in seo_list:
        new_url.append(character)

if len(new_url) == len(awesome):
    print(True)
else:
    print(False)

True


In [249]:
#seo_friendly_url_relative_length_check - url should not be longer than 120 characters      
if len(url) < 120:
    return True
else: 
    return False

In [ ]:
#Check 

if url.contains('')

In [252]:
# check the actual URL for the site (SSL Encrypted)

if 's' in req.url:
    return True
else:
    return False

https://
